In [6]:
import pandas as pd
import os
import re
import sys

#os.chdir('../')
sys.path.insert(0, '/Users/alanliu99/Documents/GitHub/yelp-recommender-system/src')
from tfidf import *

In [4]:
#os.chdir('../')
business = pd.read_csv('data/business.csv')
#review = pd.read_csv('test/testdata/test_review.csv')
las_vagas_test_reviews = pd.read_csv('test/testdata/las_vagas_test_reviews.csv')

In [7]:
user_df = preprocess_group_user_review(las_vagas_test_reviews)
rest_df = preprocess_group_restaurant_review(las_vagas_test_reviews, business)
user_feature_matrix = generate_user_matrix(user_df, 'user_id')
rest_feature_matrix = generate_user_matrix(rest_df, 'business_id')

In [35]:
las_vagas_test_reviews[las_vagas_test_reviews.business_id == random_rest][['name','business_id','city','categories']].head(1).reset_index(inplace = False)

,index,name,business_id,city,categories
0,4373,Fresh Kabob,TspQ9jCczz436RiJHJQYZw,Las Vegas,"Restaurants, Mediterranean"


In [10]:
make_recommendations(random_user, user_feature_matrix, user_df[['user_id','text', 'categories']], k=5)

,user_id,text,categories
0,TScUbGDVH5aTAQ0lHSrB5Q,Absolutely loved it Great service and exper...,"Creperies, Restaurants, Event Planning & Servi..."
1,vwLV4TkRMoghs86tB1Ke8Q,Catherine was our server and she also is one...,"Creperies, Restaurants, Event Planning & Servi..."
2,Ki6WiGwj7H6CyItdZTKjKg,This is a completely charming French lunch r...,"Creperies, Restaurants, Event Planning & Servi..."
3,8SzKrRtsiHBqR7yupCaW5Q,This is the best place for the authentic Fre...,"Creperies, Restaurants, Event Planning & Servi..."
4,e3cezqVRgQif8cyaZin1Tg,Yummy authentic and exceptional service T...,"Creperies, Restaurants, Event Planning & Servi..."


In [ ]:
las_vegas = pd.read_csv('data/Las_Vegas_user_groupby_review.csv')
#las_vegas_all_reviews = pd.read_csv('data/Las_Vegas_review.csv')
#reviews = pd.read_csv('data/reviews.csv')
business = pd.read_csv('data/business.csv')

In [ ]:
cities = ['Las Vegas','Phoenix','Toronto']
business.dropna(inplace = True)
restaurants_business = business[business.categories.str.contains("Restaurants")]
business_city = restaurants_business[restaurants_business.city == 'Las Vegas']
bus_rev_city = business_city.merge(reviews, on = 'business_id')
bus_rev_city = bus_rev_city.drop(columns = ['Unnamed: 0'])

In [ ]:
larger_than_5_review_user = las_vegas_all_reviews[las_vegas_all_reviews.user_id.isin(filtered_larger_5.index.values)]
larger_than_5_review_user

In [ ]:
larger_than_5_review_user.to_csv('5_more_review_Las_Vegas.csv', index = False)

### Preprocess for TF-IDF

In [ ]:
larger_than_5_review_user

In [ ]:
def preprocess_group_user_review(review_df):
    user_df = review_df[['text','categories','user_id']].groupby('user_id').agg({'text':list,
                                                                            'categories':lambda column: ' '.join(column)}
                                                                          ).reset_index()
    user_df.text = user_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return user_df

In [ ]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                          'stars':np.mean}
                                                                                        ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return restaurant_df.merge(business_df[['name', 'business_id']], on = 'business_id')

In [ ]:
joined_user = preprocess_group_user_review(larger_than_5_review_user)
joined_user

In [ ]:
joined_restaurant = preprocess_group_restaurant_review(larger_than_5_review_user, business)
joined_restaurant

### Convert the text review dataframe into the TF-IDF featue matrix

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(joined_user.text.apply(lambda x: x.lower()))

### User - User Simple TF-IDF and Cosine Similarity recommendation

In [ ]:
def user_recommendations(ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        User ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).head(k)

In [ ]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
user_user_df = pd.DataFrame(cosine_sim, columns=t.user_id, index=t.user_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
user_id = '---1lKK3aKOuomHnwAkAow' # input user id
add_on = t[['user_id','text', 'categories']]

user_recommendations(user_id, user_user_df, add_on, k=5)

### Resraurant - Reataurant TF-IDF approach

In [ ]:
def restaurant_recommendations(business_ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        Restaurant Business ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,business_ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(business_ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).drop_duplicates().head(k)

In [ ]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)
rest_rest_df = pd.DataFrame(cosine_sim, columns=joined_restaurant.business_id, index=joined_restaurant.business_id)

In [ ]:
# given a user ID and the similarity matrix, recommend a list of similar user
business_id = '4Nj2ktP2NjCbv2aKeH3ZBg' # input business id
add_on = a[['business_id','categories']]

restaurant_recommendations(business_id, rest_rest_df, add_on, k=5)

### Other Try

In [ ]:
for i in tqdm(range(len(cities))):
    business_city = restaurants_business[restaurants_business.city == cities[i]]
    bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
def calc_sim_user_rating(target_user_id, target_business_id, groupby_df, original_df):
    business_id_df = original_df[original_df.business_id == target_business_id].copy()
    
    target_user_tfidf = tfidf_matrix[groupby_df[groupby_df.user_id == target_user_id].index[0]].copy()
    
    dist_user = []
    for user_id in business_id_df.user_id.values:
        dist_user.append(cosine_similarity(tfidf_matrix[groupby_df[groupby_df.user_id==user_id].index[0]],
                                           target_user_vector)[0][0])
        
    business_id_df['similarity'] = dist_user    
    #Filtering technique = Calc percentile and filter percentiles > 80%
    business_id_df = business_id_df[business_id_df.rank(pct=True).similarity > .8]
     
    rating_diff = []
    for user_id in business_id_df.user_id.values:
        rating_diff.append(test_rest_user_id.stars_review[test_rest_user_id.user_id==user_id].values[0]
                            -np.average(city_train[city_train.user_id ==user_id].stars_review.values))
    test_rest_user_id['rating_diff'] = rating_diff
    
    return np.average(test_rest_user_id.rating_diff)

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in test.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], test['user_id'][i]) for i in similar_indices]

    results[row['user_id']] = similar_items[1:]
    
print('done!')

def item(id):
    return test.loc[test['user_id'] == id]['text'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")